In [1]:
"""
Convolutional Neuroscience
Accademic year 2019-2020
Homework 3

Author: Tommaso Tabarelli
Period: december 2019
"""

# Importing libraries

import argparse
import torch
import json
import re
import numpy as np
from torch.utils.data import Dataset, DataLoader
from functools import reduce
from torch import optim, nn
from network import Network, train_batch
from torch.utils.data import DataLoader
from torchvision import transforms
from pathlib import Path

In [2]:
# Defining network class

class Network(nn.Module):
    
	def __init__(self, input_size, hidden_units, layers_num, dropout_prob=0):
		# Call the parent init function (required!)
		super().__init__()
		# Define recurrent layer
		self.rnn = nn.LSTM(input_size=input_size, 
						hidden_size=hidden_units,
						num_layers=layers_num,
						dropout=dropout_prob,
						batch_first=True)
		# Define output layer
		self.out = nn.Linear(hidden_units, input_size)

	def forward(self, x, state=None):
		# LSTM
		x, rnn_state = self.rnn(x, state)
		# Linear layer
		x = self.out(x)
		return x, rnn_state
    



def train_batch(net, batch_onehot, loss_fn, optimizer):

	### Prepare network input and labels
	# Get the labels (the last letter of each sequence)
	labels_onehot = batch_onehot[:, -1, :]
	labels_numbers = labels_onehot.argmax(dim=1)
	# Remove the labels from the input tensor
	net_input = batch_onehot[:, :-1, :]
	# batch_onehot.shape =   [50, 100, 38]
	# labels_onehot.shape =  [50, 38]
	# labels_numbers.shape = [50]
	# net_input.shape =      [50, 99, 38]

	### Forward pass
	# Eventually clear previous recorded gradients
	optimizer.zero_grad()
	# Forward pass
	net_out, _ = net(net_input)

	### Update network
	# Evaluate loss only for last output
	loss = loss_fn(net_out[:, -1, :], labels_numbers)
	# Backward pass
	loss.backward()
	# Update
	optimizer.step()
	# Return average batch loss
	return float(loss.data)

In [4]:
##############################
##############################
## PARAMETERS
##############################
parser = argparse.ArgumentParser(description='Train the Blake sonnet generator network.')

# Dataset
parser.add_argument('--datasetpath', type=str, default='Songs_of_innocence.txt',
                        help='Path of the train txt file')
parser.add_argument('--crop_len',    type=int, default=100,
                        help='Number of input letters')
#parser.add_argument('--alphabet_len',   type=int,   default=,                help='Number of letters in the alphabet')

# Network
parser.add_argument('--hidden_units',   type=int,   default=128,    help='Number of RNN hidden units')
parser.add_argument('--layers_num',     type=int,   default=2,      help='Number of RNN stacked layers')
parser.add_argument('--dropout_prob',   type=float, default=0.3,    help='Dropout probability')

# Training
parser.add_argument('--batchsize',  type=int, default=154,  help='Training batch size')
parser.add_argument('--num_epochs', type=int, default=1000, help='Number of training epochs')

# Save
parser.add_argument('--out_dir', type=str, default='model', help='Where to save models and params')

_StoreAction(option_strings=['--out_dir'], dest='out_dir', nargs=None, const=None, default='model', type=<class 'str'>, choices=None, help='Where to save models and params', metavar=None)

## N.B.: to change and modify using text and not sonnet_list

In [5]:
text = open("Songs_of_innocence.txt").read()

ii = 2

sonnet_list_ = re.split('\n{5}', text)

print(len(sonnet_list_))
print(sonnet_list_[ii])

# Removing titles
#sonnet_list = list(map(lambda s: re.sub('\n?[0-9]?[[A-Z]{2,}[ -]?[A-Z]{2,}]*\n?', '', s), sonnet_list))
sonnet_list_ = list(map(lambda s: re.sub('\n?[0:9]?([A-Z]{2,}[- ]?(\'S)?){1,}\n?', '', s), sonnet_list_))

# Eliminating the first element since it's empty
sonnet_list_.pop(0)

print(len(sonnet_list_))

print("----", sonnet_list_[0], "----")

sonnet_list_ = list(map(lambda s: re.sub('\n{2,3}', '\n', s), sonnet_list_))

print("----", sonnet_list_[-1], "----")

49

THE SHEPHERD

How sweet is the shepherd’s sweet lot!
From the morn to the evening he strays;
He shall follow his sheep all the day,
And his tongue shall be fillèd with praise.

For he hears the lambs’ innocent call,
And he hears the ewes’ tender reply;
He is watchful while they are in peace,
For they know when their shepherd is nigh.
48
---- 
Piping down the valleys wild,
Piping songs of pleasant glee,
On a cloud I saw a child,
And he laughing said to me:

‘Pipe a song about a Lamb!’
So I piped with merry cheer.
‘Piper, pipe that song again.’
So I piped: he wept to hear.

‘Drop thy pipe, thy happy pipe;
Sing thy songs of happy cheer!’
So I sung the same again,
While he wept with joy to hear.


‘Piper, sit thee down and write
In a book, that all may read.’
So he vanished from my sight;
And I plucked a hollow reed,

And I made a rural pen,
And I stained the water clear,
And I wrote my happy songs
Every child may joy to hear. ----
---- 
Youth of delight! come hither
And see the openin

In [6]:
text = open("Songs_of_innocence.txt").read()

text = re.sub('\n?[0:9]?([A-Z]{2,}[- ]?(’S)?){1,}\n?', '', text)

text = text.lower()

sonnet_list = re.split('\n{5}', text)

sonnet_list = list(map(lambda s: re.sub('\n{2,3}', '\n', s), sonnet_list))

# Eliminating the first sonnet (it is only empty string)
sonnet_list.pop(0)

print(len(sonnet_list))

print(sonnet_list[1])

47

how sweet is the shepherd’s sweet lot!
from the morn to the evening he strays;
he shall follow his sheep all the day,
and his tongue shall be fillèd with praise.
for he hears the lambs’ innocent call,
and he hears the ewes’ tender reply;
he is watchful while they are in peace,
for they know when their shepherd is nigh.


In [7]:
for ii in range(len(sonnet_list)):
    print("\n\n--------------------\n\n",sonnet_list[ii], "\n\n--------------------\n\n")
        #print("\n\n\n", sonnet_list[ii], "\n----------\n", sonnet_list_[ii], end="\n\n\n\n")




--------------------

 
piping down the valleys wild,
piping songs of pleasant glee,
on a cloud i saw a child,
and he laughing said to me:
‘pipe a song about a lamb!’
so i piped with merry cheer.
‘piper, pipe that song again.’
so i piped: he wept to hear.
‘drop thy pipe, thy happy pipe;
sing thy songs of happy cheer!’
so i sung the same again,
while he wept with joy to hear.
‘piper, sit thee down and write
in a book, that all may read.’
so he vanished from my sight;
and i plucked a hollow reed,
and i made a rural pen,
and i stained the water clear,
and i wrote my happy songs
every child may joy to hear. 

--------------------




--------------------

 
how sweet is the shepherd’s sweet lot!
from the morn to the evening he strays;
he shall follow his sheep all the day,
and his tongue shall be fillèd with praise.
for he hears the lambs’ innocent call,
and he hears the ewes’ tender reply;
he is watchful while they are in peace,
for they know when their shepherd is nigh. 

-------------

In [8]:
class BlakeDataset(Dataset):
    
    def __init__(self, filepath, transform=None):
        
        ### Load data
        text = open(filepath, 'r').read()

        # Removing titles
        text = re.sub('\n?[0:9]?([A-Z]{2,}[- ]?(\'S)?){1,}\n?', '', text)

        # Lowering all text
        text = text.lower()

        # Getting the list of sonnets
        sonnet_list = re.split('\n{5}', text)

        # Removing the newlines (both 2 and 3 occurencies)
        sonnet_list = list(map(lambda s: re.sub('\n{2,3}', '\n', s), sonnet_list))

        # Eliminating the first sonnet (it is only empty string)
        sonnet_list.pop(0)

        ### Char to number
        alphabet = list(set(text))		# "set" function divides the text in single characters (not ordered)
        alphabet.sort()				# sorting not ordered characters
        
        # Deleting all non wanted characters from the alphabet: keeping only 
        #   a-z,'\n', ' ', '.', ',', '?' (the question point is charactesitic in Blake)
        alphabet_good = {'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p',
                         'q','r','s','t','u','v','w','x','y','z',' ','\n',',','.','?'}
        
        print('Found letters:', alphabet)
        print('Using only:', alphabet_good)
        # Building dictionaries
        #char_to_number = {char: number for number, char in enumerate(alphabet)}
        #number_to_char = {number: char for number, char in enumerate(alphabet)}

        char_to_number = {char: number for number, char in enumerate(alphabet_good)}
        number_to_char = {number: char for number, char in enumerate(alphabet_good)}
        
        ### Store data
        self.sonnet_list = sonnet_list
        self.transform = transform
        self.alphabet = alphabet_good
        self.char_to_number = char_to_number
        self.number_to_char = number_to_char
        
    def __len__(self):
        return len(self.sonnet_list)
        
    def __getitem__(self, idx):
        # Get sonnet text
        text = self.sonnet_list[idx]
        # Encode with numbers
        encoded = encode_text(self.char_to_number, text)

        # Create sample
        sample = {'text': text, 'encoded': encoded}
        # Transform (if defined)
        if self.transform:
            sample = self.transform(sample)
        return sample


#def encode_text(char_to_number, text):
#    encoded = [char_to_number[c] for c in text]
#    return encoded

def encode_text(char_to_number, text):
    i = -1
    for c in text:
        i+=1
        try:
            a = char_to_number[c]
        except:       # If the character is not in the char_to_number dictionary, then put commas
            s = list(text)
            s[i]=','
            text=''.join(s)
    encoded = [char_to_number[c] for c in text]
    return encoded

def decode_text(number_to_char, encoded):
    text = [number_to_char[c] for c in encoded]
    # Building proper string from a list of strings (concatenating them)
    text = reduce(lambda s1, s2: s1 + s2, text)
    return text


class RandomCrop():
    
    def __init__(self, crop_len):
        self.crop_len = crop_len

    def __call__(self, sample):
        text = sample['text']
        encoded = sample['encoded']
        # Randomly choose an index
        tot_chars = len(text)
        start_idx = np.random.randint(0, tot_chars - self.crop_len)
        end_idx = start_idx + self.crop_len
        return {**sample,
            'text': text[start_idx: end_idx],
            'encoded': encoded[start_idx: end_idx]}
        

def create_one_hot_matrix(encoded, alphabet_len):
    # Create one hot matrix
    encoded_onehot = np.zeros([len(encoded), alphabet_len])
    tot_chars = len(encoded)
    # Placing ONEs at the respective position of the letters: "encoded" indeed have numbers that encode the letters,
    # 	and here it is used as index dimension
    encoded_onehot[np.arange(tot_chars), encoded] = 1
    return encoded_onehot


class OneHotEncoder():
    
    def __init__(self, alphabet_len):
        self.alphabet_len = alphabet_len
        
    def __call__(self, sample):
        # Load encoded text with numbers
        encoded = np.array(sample['encoded'])
        # Create one hot matrix
        encoded_onehot = create_one_hot_matrix(encoded, self.alphabet_len)
        return {**sample,
            'encoded_onehot': encoded_onehot}
        
                
class ToTensor():
    
    def __call__(self, sample):
        # Convert one hot encoded text to pytorch tensor
        encoded_onehot = torch.tensor(sample['encoded_onehot']).float()
        return {'encoded_onehot': encoded_onehot}


## Preparing training the net

In [9]:
# Parse input arguments
training_args = json.load(open('my_training_args.json'))

#%% Check device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print('Selected device:', device)

dataset = BlakeDataset(filepath=training_args['datasetpath'], 
                    transform=None)

#%% Create dataset
trans = transforms.Compose([RandomCrop(training_args['crop_len']),
                    OneHotEncoder(len(dataset.alphabet)),
                    ToTensor()
                    ])

dataset = BlakeDataset(filepath=training_args['datasetpath'], 
                    transform=trans)

print("Alphabet length:", len(dataset.alphabet))

#%% Initialize network
net = Network(input_size=len(dataset.alphabet), 
            hidden_units=training_args['hidden_units'], 
            layers_num=training_args['layers_num'], 
            dropout_prob=training_args['dropout_prob'])

net.to(device)

Selected device: cpu
Found letters: ['\n', ' ', '!', '(', ')', ',', '-', '.', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'è', 'é', '—', '‘', '’', '“', '”']
Using only: {'a', 't', 'w', 'n', ' ', 'h', 'b', 'o', ',', 'l', '.', 'f', 'z', 'e', 'k', 'g', '\n', '?', 'v', 'd', 'j', 'x', 'c', 'y', 'm', 's', 'r', 'u', 'i', 'q', 'p'}
Found letters: ['\n', ' ', '!', '(', ')', ',', '-', '.', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'è', 'é', '—', '‘', '’', '“', '”']
Using only: {'a', 't', 'w', 'n', ' ', 'h', 'b', 'o', ',', 'l', '.', 'f', 'z', 'e', 'k', 'g', '\n', '?', 'v', 'd', 'j', 'x', 'c', 'y', 'm', 's', 'r', 'u', 'i', 'q', 'p'}
Alphabet length: 31


Network(
  (rnn): LSTM(31, 400, num_layers=2, batch_first=True, dropout=0.1)
  (out): Linear(in_features=400, out_features=31, bias=True)
)

In [9]:
print(training_args)

{'datasetpath': 'Songs_of_innocence.txt', 'crop_len': 50, 'hidden_units': 400, 'layers_num': 2, 'dropout_prob': 0.1, 'batchsize': 16, 'num_epochs': 1000, 'out_dir': 'model'}


## Training the net

In [10]:
#%% Train network

# Define Dataloader
dataloader = DataLoader(dataset, batch_size=training_args['batchsize'], shuffle=True, num_workers=1)
# Define optimizer
optimizer = optim.Adam(net.parameters(), weight_decay=5e-4)
# Define loss function
loss_fn = nn.CrossEntropyLoss()

# Start training
for epoch in range(int(training_args['num_epochs'])):
    print('##################################')
    print('## EPOCH '+str(epoch + 1)+"/"+str(int(training_args['num_epochs'])))
    print('##################################')
    # Iterate batches
    for batch_sample in dataloader:
        # Extract batch
        batch_onehot = batch_sample['encoded_onehot'].to(device)
        # Update network
        batch_loss = train_batch(net, batch_onehot, loss_fn, optimizer)
        print('\t Training loss (single batch):', batch_loss)

### Save all needed parameters
# Create output dir
out_dir = Path(training_args['out_dir'])
out_dir.mkdir(parents=True, exist_ok=True)
# Save network parameters
torch.save(net.state_dict(), out_dir / 'net_params.pth')

# Adding alphabet length
training_args["alphabet_len"] = len(dataset.alphabet)
# Save training parameters
with open(out_dir / 'training_args.json', 'w') as f:
    json.dump(training_args, f, indent=4)
# Save encoder dictionary
with open(out_dir / 'char_to_number.json', 'w') as f:
    json.dump(dataset.char_to_number, f, indent=4)
# Save decoder dictionary
with open(out_dir / 'number_to_char.json', 'w') as f:
    json.dump(dataset.number_to_char, f, indent=4)

##################################
## EPOCH 1/1000
##################################
	 Training loss (single batch): 3.43574595451355
	 Training loss (single batch): 3.4177017211914062
	 Training loss (single batch): 3.4129598140716553
##################################
## EPOCH 2/1000
##################################
	 Training loss (single batch): 3.395730972290039
	 Training loss (single batch): 3.352689504623413
	 Training loss (single batch): 3.4033026695251465
##################################
## EPOCH 3/1000
##################################
	 Training loss (single batch): 3.284884452819824
	 Training loss (single batch): 3.1938743591308594
	 Training loss (single batch): 2.853177785873413
##################################
## EPOCH 4/1000
##################################
	 Training loss (single batch): 2.76914119720459
	 Training loss (single batch): 3.150299072265625
	 Training loss (single batch): 3.097045421600342
##################################
## EPOCH 5/1000
###

	 Training loss (single batch): 3.0826447010040283
	 Training loss (single batch): 2.9258453845977783
##################################
## EPOCH 36/1000
##################################
	 Training loss (single batch): 2.640077590942383
	 Training loss (single batch): 3.325840473175049
	 Training loss (single batch): 3.0438756942749023
##################################
## EPOCH 37/1000
##################################
	 Training loss (single batch): 2.6155924797058105
	 Training loss (single batch): 3.377474308013916
	 Training loss (single batch): 2.806382179260254
##################################
## EPOCH 38/1000
##################################
	 Training loss (single batch): 3.0209803581237793
	 Training loss (single batch): 2.9089670181274414
	 Training loss (single batch): 3.083994150161743
##################################
## EPOCH 39/1000
##################################
	 Training loss (single batch): 2.8926596641540527
	 Training loss (single batch): 3.05186605453

	 Training loss (single batch): 2.923152446746826
	 Training loss (single batch): 2.8776488304138184
	 Training loss (single batch): 3.5029051303863525
##################################
## EPOCH 71/1000
##################################
	 Training loss (single batch): 2.901402473449707
	 Training loss (single batch): 2.777860164642334
	 Training loss (single batch): 3.1694788932800293
##################################
## EPOCH 72/1000
##################################
	 Training loss (single batch): 2.8340561389923096
	 Training loss (single batch): 3.089097023010254
	 Training loss (single batch): 2.929450750350952
##################################
## EPOCH 73/1000
##################################
	 Training loss (single batch): 3.034224510192871
	 Training loss (single batch): 3.2217793464660645
	 Training loss (single batch): 3.161283254623413
##################################
## EPOCH 74/1000
##################################
	 Training loss (single batch): 2.8424723148345

	 Training loss (single batch): 3.071009635925293
##################################
## EPOCH 105/1000
##################################
	 Training loss (single batch): 2.841610908508301
	 Training loss (single batch): 2.891688585281372
	 Training loss (single batch): 3.390174388885498
##################################
## EPOCH 106/1000
##################################
	 Training loss (single batch): 2.9298272132873535
	 Training loss (single batch): 2.8597018718719482
	 Training loss (single batch): 3.0233235359191895
##################################
## EPOCH 107/1000
##################################
	 Training loss (single batch): 2.7831063270568848
	 Training loss (single batch): 2.9165091514587402
	 Training loss (single batch): 2.6339805126190186
##################################
## EPOCH 108/1000
##################################
	 Training loss (single batch): 2.7500500679016113
	 Training loss (single batch): 2.885577917098999
	 Training loss (single batch): 2.9238288

	 Training loss (single batch): 2.480743646621704
	 Training loss (single batch): 2.6580984592437744
	 Training loss (single batch): 2.6430039405822754
##################################
## EPOCH 140/1000
##################################
	 Training loss (single batch): 2.7085721492767334
	 Training loss (single batch): 2.677532434463501
	 Training loss (single batch): 2.6795053482055664
##################################
## EPOCH 141/1000
##################################
	 Training loss (single batch): 2.626812696456909
	 Training loss (single batch): 2.3453705310821533
	 Training loss (single batch): 2.359250783920288
##################################
## EPOCH 142/1000
##################################
	 Training loss (single batch): 2.5752644538879395
	 Training loss (single batch): 2.726480722427368
	 Training loss (single batch): 3.0531272888183594
##################################
## EPOCH 143/1000
##################################
	 Training loss (single batch): 3.0276615

...

##################################
## EPOCH 862/1000
##################################
	 Training loss (single batch): 0.9642576575279236
	 Training loss (single batch): 1.0502434968948364
	 Training loss (single batch): 1.0777413845062256
##################################
## EPOCH 863/1000
##################################
	 Training loss (single batch): 1.199897289276123
	 Training loss (single batch): 0.479679673910141
	 Training loss (single batch): 0.7418422102928162
##################################
## EPOCH 864/1000
##################################
	 Training loss (single batch): 1.0976381301879883
	 Training loss (single batch): 0.9794875383377075
	 Training loss (single batch): 1.4411487579345703
##################################
## EPOCH 865/1000
##################################
	 Training loss (single batch): 0.9660755395889282
	 Training loss (single batch): 1.0467660427093506
	 Training loss (single batch): 0.7135398983955383
##################################
## 

	 Training loss (single batch): 1.0017677545547485
	 Training loss (single batch): 0.6782599687576294
	 Training loss (single batch): 1.4740432500839233
##################################
## EPOCH 897/1000
##################################
	 Training loss (single batch): 0.6852731704711914
	 Training loss (single batch): 1.2824437618255615
	 Training loss (single batch): 1.0292736291885376
##################################
## EPOCH 898/1000
##################################
	 Training loss (single batch): 0.9340311288833618
	 Training loss (single batch): 1.002034068107605
	 Training loss (single batch): 0.5809177160263062
##################################
## EPOCH 899/1000
##################################
	 Training loss (single batch): 1.1865880489349365
	 Training loss (single batch): 0.9442340731620789
	 Training loss (single batch): 0.5824597477912903
##################################
## EPOCH 900/1000
##################################
	 Training loss (single batch): 0.954

	 Training loss (single batch): 0.696546733379364
	 Training loss (single batch): 0.7589183449745178
##################################
## EPOCH 931/1000
##################################
	 Training loss (single batch): 0.7202404737472534
	 Training loss (single batch): 0.9889467358589172
	 Training loss (single batch): 1.2727348804473877
##################################
## EPOCH 932/1000
##################################
	 Training loss (single batch): 0.9630089998245239
	 Training loss (single batch): 0.48543432354927063
	 Training loss (single batch): 1.2490557432174683
##################################
## EPOCH 933/1000
##################################
	 Training loss (single batch): 0.5969839096069336
	 Training loss (single batch): 0.8695554137229919
	 Training loss (single batch): 1.3706223964691162
##################################
## EPOCH 934/1000
##################################
	 Training loss (single batch): 1.118812084197998
	 Training loss (single batch): 0.705

	 Training loss (single batch): 0.7366496920585632
##################################
## EPOCH 965/1000
##################################
	 Training loss (single batch): 0.9079659581184387
	 Training loss (single batch): 0.9221750497817993
	 Training loss (single batch): 0.5256601572036743
##################################
## EPOCH 966/1000
##################################
	 Training loss (single batch): 0.9452964067459106
	 Training loss (single batch): 0.3396050035953522
	 Training loss (single batch): 1.546496868133545
##################################
## EPOCH 967/1000
##################################
	 Training loss (single batch): 1.1138206720352173
	 Training loss (single batch): 0.7121904492378235
	 Training loss (single batch): 0.6995582580566406
##################################
## EPOCH 968/1000
##################################
	 Training loss (single batch): 0.4195690155029297
	 Training loss (single batch): 0.6566492319107056
	 Training loss (single batch): 1.594

	 Training loss (single batch): 0.32562971115112305
	 Training loss (single batch): 0.9431481957435608
	 Training loss (single batch): 0.7876369953155518
##################################
## EPOCH 1000/1000
##################################
	 Training loss (single batch): 0.7334100008010864
	 Training loss (single batch): 0.7262476086616516
	 Training loss (single batch): 0.5707294344902039


In [12]:
dataloader = DataLoader(dataset, batch_size=3, shuffle=True, num_workers=1)
for b in dataloader:
    print(b.keys())

dict_keys(['encoded_onehot'])
dict_keys(['encoded_onehot'])
dict_keys(['encoded_onehot'])
dict_keys(['encoded_onehot'])
dict_keys(['encoded_onehot'])
dict_keys(['encoded_onehot'])
dict_keys(['encoded_onehot'])
dict_keys(['encoded_onehot'])
dict_keys(['encoded_onehot'])
dict_keys(['encoded_onehot'])
dict_keys(['encoded_onehot'])
dict_keys(['encoded_onehot'])
dict_keys(['encoded_onehot'])
dict_keys(['encoded_onehot'])
dict_keys(['encoded_onehot'])
dict_keys(['encoded_onehot'])


In [11]:
### Save all needed parameters
# Create output dir
out_dir = Path(training_args['out_dir'])
out_dir.mkdir(parents=True, exist_ok=True)
# Save network parameters
torch.save(net.state_dict(), out_dir / 'net_params.pth')

# Adding alphabet length
training_args["alphabet_len"] = len(dataset.alphabet)
# Save training parameters
with open(out_dir / 'training_args.json', 'w') as f:
    json.dump(training_args, f, indent=4)
# Save encoder dictionary
with open(out_dir / 'char_to_number.json', 'w') as f:
    json.dump(dataset.char_to_number, f, indent=4)
# Save decoder dictionary
with open(out_dir / 'number_to_char.json', 'w') as f:
    json.dump(dataset.number_to_char, f, indent=4)

$$ \text{loss}(x, class) = -\log\left(\frac{\exp(x[class])}{\sum_j \exp(x[j])}\right)
                   = -x[class] + \log\left(\sum_j \exp(x[j])\right) $$

In [12]:
##############################
##############################
## PARAMETERS
##############################
#parser = argparse.ArgumentParser(description='Generate sonnet starting from a given text')

#parser.add_argument('--sonnet_seed', type=str, default='the', help='Initial text of the sonnet')
#parser.add_argument('--model_dir',   type=str, default='pretrained_models/model_3', help='Network model directory')

##############################
##############################
##############################

### Parse input arguments
#args = parser.parse_args()

model_dir = Path("model_Blake_2019-12-03_22:22/")
sonnet_seed = "burning tiger little lamb"

#%% Load training parameters
model_dir = Path("model_Blake_2019-12-10_09:13")
print ('Loading model from: %s' % model_dir)
training_args = json.load(open(model_dir / 'training_args.json'))

#%% Load encoder and decoder dictionaries
number_to_char = json.load(open(model_dir / 'number_to_char.json'))
char_to_number = json.load(open(model_dir / 'char_to_number.json'))

#%% Initialize network
net = Network(input_size=training_args['alphabet_len'], 
            hidden_units=training_args['hidden_units'], 
            layers_num=training_args['layers_num'])

#%% Load network trained parameters
net.load_state_dict(torch.load(model_dir / 'net_params.pth', map_location='cpu'))
net.eval() # Evaluation mode (e.g. disable dropout)

#%% Find initial state of the RNN
with torch.no_grad():
    # Encode seed
    seed_encoded = encode_text(char_to_number, sonnet_seed)
    # One hot matrix
    seed_onehot = create_one_hot_matrix(seed_encoded, training_args['alphabet_len'])
    # To tensor
    seed_onehot = torch.tensor(seed_onehot).float()
    # Add batch axis
    seed_onehot = seed_onehot.unsqueeze(0)
    # Forward pass
    net_out, net_state = net(seed_onehot)
    # Get the most probable last output index
    # ---------- sampling using softmax ----------
    next_char_encoded = net_out[:, -1, :].argmax().item()
    # Print the seed letters
    print(sonnet_seed, end='', flush=True)
    print(number_to_char[str(next_char_encoded)])

#%% Generate sonnet
new_line_count = 0
tot_char_count = 0
while True:
    with torch.no_grad(): # No need to track the gradients
        # The new network input is the one hot encoding of the last chosen letter
        net_input = create_one_hot_matrix([next_char_encoded], len(dataset.alphabet))
        net_input = torch.tensor(net_input).float()
        net_input = net_input.unsqueeze(0)
        # Forward pass
        net_out, net_state = net(net_input, net_state)
        # Get the most probable letter index
        distrib = np.array(nn.functional.softmax(net_out, dim=-1))
        next_char_encoded = np.random.choice(len(distrib.ravel()), size=1, p=distrib.ravel())[0]
        
        #next_char_encoded = net_out.argmax().item()
        
        # Decode the letter
        next_char = number_to_char[str(next_char_encoded)]
        print(next_char, end='', flush=True)
        # Count total letters
        tot_char_count += 1
        # Count new lines
        if next_char == '\n':
            new_line_count += 1
        # Break if 14 lines or 2000 letters
        if new_line_count == 14 or tot_char_count > 2000:
            break

Loading model from: model_Blake_2019-12-10_09:13
burning tiger little lambe
s,ade, hererat n wist lolen,
and npghe gillad,wafw,bey iroen,
in the bayadtawhe hey roarn,
fitl woat dfskeog,
leve yiad the ly is aud a the s and afd whot oread,
du csangheef he wheakirilgs,
and hath foren,
wiu beekin the merrill.d,
ezy onaptoeg theu nothen,, and hearttor iso ssark,
fu mrce cislds,f, cton t ored ond hapt feep,
iw e ghe sill thoue deasn,
an the haghlene he desal.
anvt bo if feev,
and boslofu woagn deed,


In [12]:
print(training_args)

{'datasetpath': 'Songs_of_innocence.txt', 'crop_len': 50, 'hidden_units': 400, 'layers_num': 2, 'dropout_prob': 0.1, 'batchsize': 16, 'num_epochs': 1000, 'out_dir': 'model', 'alphabet_len': 31}
